In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler

from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
import itertools
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cuda


In [5]:
NUM_EPOCHS = 80
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
NUM_CLASSES = 10      # FashionMNIST has 10 classes
INPUT_CHANNELS = 1    # FashionMNIST is grayscale

In [6]:
image_transforms = transforms.Compose([
    transforms.Resize(224), # ResNet-18 standard input size
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5], # Grayscale mean
        std=[0.5]   # Grayscale std
    ),
])

In [7]:
# Training
train_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=image_transforms
)
# Testing
test_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=image_transforms
)

# Data Loader
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=5
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=5
)

100%|██████████| 26.4M/26.4M [00:36<00:00, 721kB/s] 
100%|██████████| 29.5k/29.5k [00:00<00:00, 178kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.09MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 7.73MB/s]


In [8]:
def train_model(model, train_loader, criterion, optimizer, num_epochs, device):
    """Handles the training process and evaluates after each epoch, tracking losses."""
    model.train()
    print("\nStarting training on FashionMNIST...")

    # Clear logs before starting
    train_losses, test_losses, test_accuracies=[],[],[]

    for epoch in range(num_epochs):
        total_loss = 0.0
        total_samples = 0

        # Training Loop
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # === sample-weighted loss, matches evaluation ===
            batch_size = labels.size(0)
            total_loss += loss.item() * batch_size
            total_samples += batch_size

        # === Correct per-sample training loss ===
        epoch_loss = total_loss / total_samples
        train_losses.append(epoch_loss)

        # Evaluate on test set
        acc, test_loss = evaluate_model(model, test_loader, criterion, device)

        test_losses.append(test_loss)
        test_accuracies.append(acc)

        print(f"Epoch {epoch+1} finished. Train Loss: {epoch_loss:.4f} | Test Loss: {test_loss:.4f} | Test Accuracy: {acc:.2f}%")
    return train_losses, test_losses, test_accuracies


def evaluate_model(model, test_loader, criterion, device):
    """Calculates the Top-1 accuracy AND the average test loss."""
    model.eval()
    correct_predictions = 0
    total_samples = 0
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            batch_size = labels.size(0)
            total_loss += loss.item() * batch_size
            total_samples += batch_size

            _, predicted = outputs.max(1)
            correct_predictions += (predicted == labels).sum().item()

    accuracy = 100 * correct_predictions / total_samples
    avg_loss = total_loss / total_samples

    return accuracy, avg_loss


In [9]:
class SimpleBilinearCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),           # -> 14x14
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2)            # -> 7x7
        )
        # Bilinear pooling output: 256x256 features
        self.fc = nn.Linear(256 * 256, num_classes)
        
    def forward(self, x):
        x = self.features(x)  # [B, 256, 7, 7]
        B, C, H, W = x.size()
        x = x.view(B, C, H * W)
        # Bilinear pooling
        x = torch.bmm(x, x.transpose(1, 2)) / (H * W)  # [B, C, C]
        x = x.view(B, -1)
        x = torch.sign(x) * torch.sqrt(torch.abs(x) + 1e-10)  # signed sqrt
        x = F.normalize(x)
        return self.fc(x)

model= SimpleBilinearCNN().to(DEVICE)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=LEARNING_RATE,weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=NUM_EPOCHS, gamma=0.1)
import gc
gc.collect()
torch.cuda.empty_cache()
train_losses, test_losses, test_accuracies = train_model(
    model, train_loader, criterion, optimizer, NUM_EPOCHS, DEVICE
)



Starting training on FashionMNIST...


Epoch 1/80: 100%|██████████| 1875/1875 [02:35<00:00, 12.09it/s]


Epoch 1 finished. Train Loss: 1.3349 | Test Loss: 0.9360 | Test Accuracy: 68.26%


Epoch 2/80: 100%|██████████| 1875/1875 [03:45<00:00,  8.32it/s]


Epoch 2 finished. Train Loss: 0.7856 | Test Loss: 0.7112 | Test Accuracy: 76.17%


...

Epoch 78 finished. Train Loss: 0.0537 | Test Loss: 0.3923 | Test Accuracy: 87.35%


Epoch 79/80: 100%|██████████| 1875/1875 [03:45<00:00,  8.30it/s]


Epoch 79 finished. Train Loss: 0.0504 | Test Loss: 0.3908 | Test Accuracy: 87.57%


Epoch 80/80: 100%|██████████| 1875/1875 [03:45<00:00,  8.31it/s]


Epoch 80 finished. Train Loss: 0.0485 | Test Loss: 0.3980 | Test Accuracy: 87.28%


In [11]:
import pandas as pd
df = pd.DataFrame({
        "epoch": list(range(1, len(train_losses) + 1)),
        "train_loss": train_losses,
        "test_loss": test_losses,
        "test_accuracy": test_accuracies,
    })
df.to_csv("BCNN_mnist.csv")